# Habituation with IBCM neurons gating inhibitory neurons
The details of the model are described in other Jupyter notebooks (e.g., ibcm_inhibition_three_components.ipynb). The goal here is to include this model in the full olfactory network down to Kenyon cells, apply it to increasingly realistic olfactory backgrounds and estimate its performance at 1) inhibiting the fluctuating background, and 2) still recognizing new odors. 

Here, in particular, we focus on log-normal concentration fluctuations. 

In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from time import perf_counter

from utils.statistics import seed_from_gen
from modelfcts.ibcm import (
    integrate_inhib_ibcm_network, 
    integrate_inhib_ibcm_network_tanh,
    relu_inplace, 
    compute_mbars_cgammas_cbargammas
)
# Functions to update the fluctuating background variable
from modelfcts.backgrounds import (
    update_logou_kinputs, 
    decompose_nonorthogonal_basis, 
    generate_odorant, 
    logof10
)

# Run a simulation
Hopefully, the inhibitory neurons can be combined to perfectly inhibit the input. 

In [ ]:
### General simulation parameters
n_dimensions = 4  # Half the real number for faster simulations
# The larger the dimension, the more likely the odors are orthogonal. 
n_components = 3  # no need to look at super complicated odors for now; keep effective space 3D
# Can actually look at this latent space by Gram-Schmidt to find orthogonal axes spanning the input odors. 
n_neurons = 12  # Start small

# Simulation times
duration = 160000.0
deltat = 1.0
learnrate = 0.0015
tau_avg = 200
coupling_eta = 0.05 / n_neurons
saturation_ampli = 50.0

inhib_rates = [0.00025, 0.00005]  # alpha, beta
ibcm_rates = [learnrate, tau_avg, coupling_eta, saturation_ampli]

# Symmetric components to begin with
back_components = 0.1*np.ones([n_components, n_dimensions])
for i in range(n_components):
    if i < n_dimensions:
        back_components[i, i] = 0.8
    else:  # If there are more components than there are dimensions (ORNs)
        back_components[i, i % n_dimensions] = 0.8 - i
    # Normalize
    back_components[i] = back_components[i] / np.sqrt(np.sum(back_components[i]**2))
print(back_components)

# Initial synaptic weights: small positive noise near origin
# Issue: the neurons do not seem to distribute equally to the 3 fixed points with this choice.
# Maybe because of lack of symmetry there are not 3 stable fixed points anymore? Tricky. 
# Try other initial conditions
rgen_meta = np.random.default_rng(seed=0x959905bd65b43006c10a3b72fb9ab60f)
#init_synapses = 0.1*rgen_meta.random(size=[n_neurons, n_dimensions])
init_synapses = 0.5*rgen_meta.random(size=[n_neurons, n_components]).dot(back_components)

# Try forcing a third of the neurons to each fixed point
# By initializing them orthogonal to two of three background components. 
force_init = False
if force_init:
    init_synapses = np.zeros([n_neurons, n_components])
    orthogonal1 = np.cross(back_components[0], back_components[1])
    orthogonal2 = np.cross(back_components[1], back_components[2])
    orthogonal3 = np.cross(back_components[2], back_components[0])
    init_synapses[:n_neurons // 3] = orthogonal1[np.newaxis, :]
    init_synapses[n_neurons // 3:2*n_neurons//3] = orthogonal2[np.newaxis, :]
    init_synapses[2*n_neurons//3:] = orthogonal3[np.newaxis, :]
    init_synapses += (rgen_meta.random(size=[n_neurons, n_components]) - 0.5) * 0.25

# Initial background vector and initial nu values
# Log-normal concentrations, nus are the logs of concentrations
averages_nu = -0.5*np.ones(n_components)  # Average of log(c); for c < 1, these averages are < 0
init_nu = averages_nu.copy()
init_bkvec = np.exp(averages_nu*logof10).dot(back_components)
# nus are first in the list of initial background params
init_back_list = [init_nu, init_bkvec]

## Compute the matrices in the Ornstein-Uhlenbeck update equation
# Update matrix for the mean term: 
# Exponential decay with time scale tau_nu over time deltat
tau_nu = 2.0  # Fluctuation time scale of the background nu_alphas (same for all)
update_mat_A = np.identity(n_components)*np.exp(-deltat/tau_nu)

# Steady-state covariance matrix
sigma2 = 0.09
correl_rho = 0.0
steady_covmat = correl_rho * sigma2 * np.ones([n_components, n_components])  # Off-diagonals: rho
steady_covmat[np.eye(n_components, dtype=bool)] = sigma2  # diagonal: ones

# Mean and variance of the concentrations themselves
# Using moments of log-normal: https://en.wikipedia.org/wiki/Log-normal_distribution
mean_nu_lnbase = averages_nu.mean() * logof10
vari_nu_lnbase = sigma2 * logof10**2
lognorm_mean = np.exp(mean_nu_lnbase + vari_nu_lnbase/2.0)
lognorm_vari = (np.exp(vari_nu_lnbase) - 1.0)*np.exp(2*mean_nu_lnbase + vari_nu_lnbase)
# Third centered moment: rom skewness, multiply by its variance**3
lognorm_skewness = (np.exp(vari_nu_lnbase) + 2)*np.sqrt(np.exp(vari_nu_lnbase) - 1)
lognorm_thirdmom = lognorm_skewness * lognorm_vari**1.5

# Cholesky decomposition of steady_covmat gives sqrt(tau/2) B
# Update matrix for the noise term: \sqrt(tau/2(1 - exp(-2*deltat/tau))) B
psi_mat = np.linalg.cholesky(steady_covmat)
update_mat_B = np.sqrt(1.0 - np.exp(-2.0*deltat/tau_nu)) * psi_mat

back_params = [update_mat_A, update_mat_B, back_components, averages_nu]

In [ ]:
# init_synapses, update_ou_kinputs, init_back_list, ibcm_rates, 
# inhib_rates, back_params, duration, deltat, seed=seed_from_gen(rgen_meta), noisetype="normal"
#init_synapses = mser[-1]
sim_results = integrate_inhib_ibcm_network_tanh(init_synapses, update_logou_kinputs, init_back_list, 
                    ibcm_rates, inhib_rates, back_params, duration, deltat, 
                    seed=seed_from_gen(rgen_meta), noisetype="normal")
# tseries, bk_series, bkvec_series, m_series, cbar_series, w_series, s_series
tser, nuser, bkvecser, mser, cbarser, _, wser, sser = sim_results

### Background statistics
Useful figure for the paper

In [ ]:
# All odors have the same statistics, flatten before taking histogram
odor_concs_ser = np.exp((nuser+averages_nu[None, :])*logof10)
odor_concs_histo, odor_concs_binseps = np.histogram(odor_concs_ser, bins=100, density=True)

# Plot histogram
fig, ax = plt.subplots()
ax.bar((odor_concs_binseps[1:]+odor_concs_binseps[:-1])/2.0, odor_concs_histo, 
       width=np.diff(odor_concs_binseps), color="grey")
ax.set(xlabel="Odor concentration", ylabel="Probability density", yscale="log")
plt.show()
plt.close()

### Check the synaptic weights against fixed points
The analytical prediction neglecting correlations between $\vec{m}$ and $\nu$ is verified, provided that the time scales $\tau_{\nu}$ and $\frac{1}{\mu}$ are different enough. Computing corrections to account for incompletely separated time scales would be very hard, since the equation for $\vec{m}$ is a multivariate, non-linear stochastic differential equation. 

In [ ]:
from modelfcts.ibcm_analytics import fixedpoint_thirdmoment_perturbtheory, fixedpoint_thirdmoment_exact
from simulfcts.plotting import plot_cbars_gammas_sums, plot_cbars_gamma_series, plot_3d_series, plot_w_matrix

In [ ]:
# Calculate cgammas_bar and mbars
transient = 120000
# Dot products \bar{c}_{\gamma} = \bar{\vec{m}} \cdot \vec{x}_{\gamma}
mbarser, c_gammas, cbars_gamma = compute_mbars_cgammas_cbargammas(mser, coupling_eta, back_components)
# Compute analytical prediction for sum of cgammas and (cgamma squared)s. 
# Uses perturbation theory even though the third moment isn't exactly small
res = fixedpoint_thirdmoment_perturbtheory(lognorm_mean, lognorm_vari, lognorm_thirdmom, 
                                           1, n_components-1, m3=1.0, order=1)

pred_cbars_gamma = res[:2]
pred_sums_cbars = list(res[2:])
# The function returns c_d, which is sum of c_gammas times average concentration
pred_sums_cbars[0] = pred_sums_cbars[0] / lognorm_mean
pred_sums_cbars = tuple(pred_sums_cbars)

# Compare to the exact solution
res = fixedpoint_thirdmoment_exact([lognorm_mean, lognorm_vari, lognorm_thirdmom], 1, n_components-1)

pred_cbars_gamma_exact = res[:2]
pred_sums_cbars_exact = list(res[2:])
# The function returns c_d, which is sum of c_gammas times average concentration
pred_sums_cbars_exact[0] = pred_sums_cbars_exact[0] / lognorm_mean
pred_sums_cbars_exact = tuple(pred_sums_cbars_exact)

sums_cbars_gamma = np.sum(cbars_gamma, axis=2)
sums_cbars_gamma2 = np.sum(cbars_gamma*cbars_gamma, axis=2)

# Constaint 1: sum of c_gammas for each neuron, equal to 1 plus correction
print("Comparison to analytical fixed points")
print("This should be approximately zeros:", np.mean(sums_cbars_gamma[transient:], axis=0) / pred_sums_cbars[0] - 1.0)
print("This should be all zeros, exact analytical solution:", np.mean(sums_cbars_gamma[transient:], axis=0) / pred_sums_cbars_exact[0] - 1.0)
# Constraint 2: sum of c_gammas^2 for each neuron, compared to 1/sigma^2 + correction
print("This should be all approximately zeros:", np.mean(sums_cbars_gamma2[transient:], axis=0) / pred_sums_cbars[1] - 1.0)
print("This should be all zeros, exact analytical solution:", np.mean(sums_cbars_gamma2[transient:], axis=0) / pred_sums_cbars_exact[1] - 1.0)

In [ ]:
fig, axes = plot_cbars_gammas_sums(tser, sums_cbars_gamma, sums_cbars_gamma2, skp=200, skp_lbl=1)
axes[0].axhline(pred_sums_cbars[0], ls="--", color="k", label=r"Perturb. $1 / \langle \nu \rangle$")
axes[1].axhline(pred_sums_cbars[1], ls="--", color="k", label=r'Perturb. $1 / \sigma^2$')
axes[0].axhline(pred_sums_cbars_exact[0], ls="-.", color="grey", label=r"Exact $1 / \langle \nu \rangle$")
axes[1].axhline(pred_sums_cbars_exact[1], ls="-.", color="grey", label=r'Exact $1 / \sigma^2$')
for ax in axes:
    ax.get_legend().set_visible(False)
axes[0].legend(*[a[-2:] for a in axes[0].get_legend_handles_labels()])
axes[1].legend(*[a[-2:] for a in axes[1].get_legend_handles_labels()])
# fig.savefig("figures/three_odors/sum_cgammas_squared_lognormal_background.pdf", transparent=True)
plt.show()
plt.close()

In [ ]:
# Surprisingly good!
fig, ax, _ = plot_cbars_gamma_series(tser, cbars_gamma, skp=100, transient=50000)
ax.get_legend().set_visible(False)

# Annotate with analytical prediction. Might fail because third moment is high. 
ax.axhline(pred_cbars_gamma[0], ls="--", color="k", label=r"Perturbative $\bar{c}_{\gamma=\mathrm{specific}}$")  # higher value
ax.axhline(pred_cbars_gamma[1], ls=":", color="k", label=r"Perturbative $\bar{c}_{\gamma=\mathrm{non}}$")  # lower value
# Exact solution should align well nevertheless
ax.axhline(pred_cbars_gamma_exact[0], ls="--", color="grey", label=r"Exact $\bar{c}_{\gamma=\mathrm{specific}}$")  # higher value
ax.axhline(pred_cbars_gamma_exact[1], ls=":", color="grey", label=r"Exact $\bar{c}_{\gamma=\mathrm{non}}$")  # lower value
ax.legend(loc="upper left")
plt.show()
plt.close()

In [ ]:
fig, ax = plot_3d_series(mbarser, dim_idx=[0, 1, 2], transient=10000, skp=1000)

# Annotate with vectors representing the odor components
orig = np.zeros([n_components, n_components])
xlim, ylim, zlim = ax.get_xlim(), ax.get_ylim(), ax.get_zlim()
scale = 3
vecs = back_components.copy()
for i in range(n_components):
    vecs[i] = back_components[i] / np.sqrt(np.sum(back_components[i]**2)) * scale
ax.quiver(*orig, *(vecs[:, :3].T), color="k", lw=2.0)
ax.view_init(azim=45, elev=30)
ax.set(xlabel=r"$\overline{m}_1$", ylabel=r"$\overline{m}_2$", zlabel=r"$\overline{m}_3$")
# fig.savefig("figures/three_odors/points_fixes_ibcm_3_odeurs_lognormal.pdf", transparent=True)
plt.show()
plt.close()

## Evolution of the inhibitory neurons' weights $\vec{w}_i$
Analytically, I find that, on average, $\vec{w}_i$ converges to $\vec{x}(\pm \sigma)$, i.e. to either input vector one standard deviation away from the mean input. So, here, I compare the numerical results for $\vec{w}$ to the possible fixed points. 

In [ ]:
# Plotting the time course of the dot products -- not interesting with gaussian degeneracy
# Unclear what it shows. 
fig, axes = plot_w_matrix(tser, wser, skp=500, lw=1.5)
        
plt.show()
plt.close()

## Background before and after inhibition

In [ ]:
from simulfcts.plotting import plot_background_norm_inhibition, plot_background_neurons_inhibition

In [ ]:
fig, ax, bknorm_ser, snorm_ser = plot_background_norm_inhibition(tser, bkvecser, sser)

# Compute noise reduction factor, annotate
transient = 50000
avg_bknorm = np.mean(bknorm_ser[transient:])
avg_snorm = np.mean(snorm_ser[transient:])
avg_reduction_factor = avg_snorm / avg_bknorm
std_bknorm = np.std(bknorm_ser[transient:])
std_snorm = np.std(snorm_ser[transient:])
std_reduction_factor = std_snorm / std_bknorm

print("Mean activity norm reduced to "
      + "{:.1f} % of input".format(avg_reduction_factor * 100))
print("Standard deviation of activity norm reduced to "
      + "{:.1f} % of input".format(std_reduction_factor * 100))
ax.annotate("St. dev. reduced to {:.1f} %".format(std_reduction_factor * 100), 
           xy=(0.98, 0.98), xycoords="axes fraction", ha="right", va="top")

ax.legend(loc="center right", bbox_to_anchor=(1.0, 0.8))
fig.tight_layout()
fig.savefig("figures/three_odors/inhibition_lognormal_background_norm_3odors.pdf", 
            transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
fig, axes_mat, axes = plot_background_neurons_inhibition(tser, bkvecser, sser)
axes[-1].legend(loc="center right", bbox_to_anchor=(1.0, 0.6), fontsize=8, handlelength=1.5)
fig.tight_layout()
#fig.savefig("figures/three_odors/inhibition_lognormal_background_neurons_3odors.pdf", 
#            transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
# 3D plot of the original and inhibited odors, sampled sparsely in time
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Raw background
skp = 1000
tslice = slice(transient, None, skp)
ax.scatter(bkvecser[tslice, 0], bkvecser[tslice, 1], bkvecser[tslice, 2], color="r", label="Background")
# Compare to inhibition of the mean
mean_inhibition = bkvecser - np.mean(bkvecser[transient:], axis=0)*inhib_rates[0]/sum(inhib_rates)
ax.scatter(mean_inhibition[tslice, 0], mean_inhibition[tslice, 1], mean_inhibition[tslice, 2], 
           color="xkcd:light blue", label="Average subtraction")
ax.scatter(sser[tslice, 0], sser[tslice, 1], sser[tslice, 2], 
           color="b", label="IBCM inhibition")
ax.scatter(0, 0, 0, color="k", s=200, alpha=1)
ax.view_init(azim=160, elev=20)
ax.legend(loc="upper left", bbox_to_anchor=(1, 0.85))
plt.show()
plt.close()

# Save some results for re-plotting
Use as a sample simulation of a log-normal background. 

In [ ]:
results_dir = "results/for_plots/"
np.savez_compressed(results_dir + "sample_lognormal_simulation.npz", nuser=nuser)

# Adding correlation between odors
Not sure what happens then. Analytical predictions fail for non-gaussian distributions. For gaussian, this amounts to a re-definition of $\vec{x}_{\gamma}$s, but extra third moment terms appear otherwise. 

In [ ]:
rgen_meta_cr = np.random.default_rng(seed=0x91117e6a405e752c66db7f05127e03a6)
#init_synapses = 0.1*rgen_meta.random(size=[n_neurons, n_dimensions])
init_synapses_cr = 0.5*rgen_meta_cr.random(size=[n_neurons, n_components]).dot(back_components)

# Steady-state covariance matrix
sigma2_cr = 0.09
correl_rho_cr = 0.5
steady_covmat_cr = correl_rho_cr * sigma2_cr * np.ones([n_components, n_components])  # Off-diagonals: rho
steady_covmat_cr[np.eye(n_components, dtype=bool)] = sigma2_cr  # diagonal: ones

# Mean and variance of the concentrations themselves
# Using moments of log-normal: https://en.wikipedia.org/wiki/Log-normal_distribution
# Not sure what they are when correlated; TODO

# Cholesky decomposition of steady_covmat gives sqrt(tau/2) B
# Update matrix for the noise term: \sqrt(tau/2(1 - exp(-2*deltat/tau))) B
psi_mat_cr = np.linalg.cholesky(steady_covmat_cr)
update_mat_B_cr = np.sqrt(1.0 - np.exp(-2.0*deltat/tau_nu)) * psi_mat_cr

back_params_cr = [update_mat_A, update_mat_B_cr, back_components, averages_nu]

In [ ]:
# init_synapses, update_ou_kinputs, init_back_list, ibcm_rates, 
# inhib_rates, back_params, duration, deltat, seed=seed_from_gen(rgen_meta), noisetype="normal"
#init_synapses = mser[-1]
sim_results = integrate_inhib_ibcm_network_tanh(init_synapses_cr, update_logou_kinputs, init_back_list, 
                    ibcm_rates, inhib_rates, back_params_cr, duration, deltat, 
                    seed=seed_from_gen(rgen_meta_cr), noisetype="normal")
# tseries, bk_series, bkvec_series, m_series, cbar_series, w_series, s_series
tser_cr, nuser_cr, bkvecser_cr, mser_cr, cbarser_cr, _, wser_cr, sser_cr = sim_results

In [ ]:
# Visualize correlations between odors
# All odors have the same statistics, flatten before taking histogram
odor_concs_ser_cr = np.exp((nuser_cr+averages_nu[None, :])*logof10)

# Plot a time slice
tslice = slice(0, 200, 1)
fig, ax = plt.subplots()
odor_colors = sns.color_palette("Greys", n_colors=n_components)
for i in range(n_components):
    ax.plot(tser_cr[tslice], odor_concs_ser_cr[tslice, i], label=r"$\gamma = {}$".format(i), 
            color=odor_colors[i])
ax.set(xlabel="Time (steps)", ylabel="Odor concentration")
ax.legend()
fig.tight_layout()
plt.show()
plt.close()

In [ ]:
# Calculate cgammas_bar and mbars
transient = 120000
# Dot products \bar{c}_{\gamma} = \bar{\vec{m}} \cdot \vec{x}_{\gamma}
mbarser_cr, c_gammas_cr, cbars_gamma_cr = compute_mbars_cgammas_cbargammas(mser_cr, coupling_eta, back_components)

# Analytical predictions already computed, not assuming correlations 
# change anything -- to see how wrong it is to suppose that
sums_cbars_gamma_cr = np.sum(cbars_gamma_cr, axis=2)
sums_cbars_gamma2_cr = np.sum(cbars_gamma_cr**2, axis=2)

In [ ]:
# The analytical prediction is quantitatively off, but the principle holds
# Each neuron becomes specific to one $\vec{x}_{\gamma}$, non-specific to others. 
fig, ax, _ = plot_cbars_gamma_series(tser_cr, cbars_gamma_cr, skp=100, transient=50000)
ax.get_legend().set_visible(False)

# Annotate with analytical prediction. Might fail because third moment is high. 
ax.axhline(pred_cbars_gamma[0], ls="--", color="k", label=r"Analytic $\bar{c}_{\gamma=\mathrm{specific}}$")  # higher value
ax.axhline(pred_cbars_gamma[1], ls=":", color="grey", label=r"Analytic $\bar{c}_{\gamma=\mathrm{non-specif}}$")  # lower value
ax.legend(loc="upper left")
plt.show()
plt.close()

# Understanding why neurons fail to distribute to different fixed points
# THIS IS NOW OUTDATED, THE NETWORK WORKS
Because of optimally-derived $W$ weights, all is fine. 

Still, IBCM neurons may not distribute optimally, should double-check TODO

One hypothesis: with the log-normal noise, basins of attractor favour, in high-dimensional space ($K\times N$), steady-states where the neurons go to the same fixed point. Not sure why that would be and hard to understand. 
 - Consequence of that hypothesis: the neurons would go to the same fixed point if they are initialized at the same places initially, no matter the realization of the process. 


Another hypothesis (Gautam Reddy): a large fluctuation along one of the components early in the simulation pushes all neurons in the basin of attraction of the same fixed point. 
 - Consequence of that hypothesis: even initialized at the same initial positions, neurons would converge to different fixed points from one realization to the next. 
 
Note: to test both hypotheses, we need to use one seeded random generator for initial positions, and another generator for the time simulations. We keep the initial position generator at the same seed, such that we can keep the same initial conditions, but we try different seeds for the time evolution process, to obtain different dynamical realizations.

If the majority fixed point changes from one realization to the next, starting from the same initial conditions, the second hypothesis is more likely. I should then try to identify the fluctuation that causes neurons to change basin of attraction. 

However, if the final fixed point is always the same, it means the cause is not in a realization-dependent fluctuation, but really in the structure of the model, averaged over realizations. 

## Code to achieve this test

Anyways, the first thing I need is a fixed point detector, to detect automatically which fixed point neurons converged to. I coded such a function below; it is based on the analytical fixed points. A simple clustering algorithm could work too when I start working on cases for which I have absolutely no analytical answer. 

When I start working with more than three components, I will be able to check. But here, I am sticking to three components, where obviously the dot product with two components will be one value, and with the third, another. 
 - Note that it could be that the dot product is the same for all components, or that two components have the high positive dot product, the odd one being at the negative dot product. But that isn't the case. Numerically, all neurons have two dot products equal to the low value, and one equal to the high value. There must be some analytical proof of that... 
 
 ## Analytical calculations to come
 - Stability of fixed points
 - Multiscale analysis to understand where the time scale of convergence of inhibitory neurons come from. Even get an expression for it in terms of the model rates. See Misbah and Bender-Orszag. Or slow-fast analysis? See papers from Prof. Khadra's course. 
 - WORK HARD!!! Stop feeling that I don't master the system well enough and hence shying away from calculations. This is a vicious circle that I need to break by trying as hard as I can to do those calculations. Set long time slots aside to do that work. 

In [ ]:
# Run a few simulations with different seeds
# Record the late time series of synaptic weights \bar{m}, discarding transient time steps. 
late_time_series = []
noise_reductions = []
n_simuls = 10
last_steps = 10000
for i in range(n_simuls):
    sim_results = integrate_inhib_ibcm_network(init_synapses, update_logou_kinputs, init_back_list, inhib_rates,
                    back_params, duration-20000, deltat, learnrate=learnrate, seed=50981*(i+1), 
                    noisetype="normal", tavg=tau_avg, coupling=coupling_eta)
    late_time_series.append(sim_results[1][-last_steps:])
    # Also compute the noise reduction
    inhib_projser = apply_background_inhibition(sim_results[4], sim_results[6], sim_results[7], n_neurons)
    stdev_inhib_comps = np.std(inhib_projser[-last_steps:], axis=0)
    stdev_bk_comps = np.std(sim_results[7][-last_steps:], axis=0)
    noise_reductions.append((stdev_inhib_comps/stdev_bk_comps)*100)
    print("Finished simulation {}; noise reduction by {} %".format(i, noise_reductions[-1]))

In [ ]:
def annotate_fixed_point(neuron_ser, fixed_pts_dict={}, distance_metric="euclidean"):
    """ Given the steady-state time series of a neuron, determine which point it
    is closest to in the dictionary of possible fixed points, fixed_pts_dict. 
    Even if the analytical fixed point predictions are not too great, as long
    as they are closest to the real fixed points than to one another, this should
    be good enough to distinguish neurons that converged to different fixed points. 
    
    Args:
        neuron_ser (np.ndarray): shape (n_time_steps, n_dimensions). 
        fixed_pts_dict (dict): each value is a potential fixed point, 
            a 1d array of shape (n_dimensions,), and the key of that value
            is the "annotation" of that fixed point.
        distance_metric (str): currently, only "euclidean" is supported. 
            Could use sklearn's metrics to allow more. Not implemented for now. 
            
    Returns:
        annotation (object): the key of the fixed point in fixed_pts_dict that
            is closest to the average steady-state of the neuron. 
        distance (float): the distance to the closest fixed point. 
    """
    if distance_metric != "euclidean":
        raise NotImplementedError("Currently, only 'euclidean' distance metric is supported.")
    avg_neuron = np.mean(neuron_ser, axis=0)
    distances = {}
    for tag in fixed_pts_dict.keys():
        distances[tag] = np.sqrt(np.sum((avg_neuron - fixed_pts_dict[tag])**2))
    annotate = max(distances, key=distances.get)
    return annotate, distances[annotate]
        

In [ ]:
def approx_fixed_points(backvecs, avg_nus):
    """ As a very crude approximation of the fixed points, instead of going through
    the perturbative solution, use vectors orthogonal to all but one of the 
    background components. Using Gram-Schmidt, then give it a norm of 1/<nu>. 
    Args:
        backvecs (np.ndarray): array of background components, 
            indexed (component, dimension)
        avg_nus (np.ndarray): array of average nu of each background vector. 
    """
    # Minimize backvecs.dot(B) - identity, solving for B, the new basis
    # Identity should have as many rows as there are vectors, so first dim. of backvecs. 
    vecs, _, _, _ = np.linalg.lstsq(backvecs, np.identity(backvecs.shape[0]), rcond=None)
    # Normalize columns, then multiply by 1/nu
    vecs = vecs / np.sqrt(np.sum(vecs**2, axis=0))
    vecs = vecs / avg_nus
    # Return transposed, so each row contains a vector
    return vecs.T

In [ ]:
# Find the fixed point to which each neuron converged. 
# Determine approximate fixed points as a guideline. 
# The averages are 10^<nu>
orthogonal_fixed_points = approx_fixed_points(back_components, np.exp(logof10*averages_nu))
print(orthogonal_fixed_points)
dict_fixed_points = {i:orthogonal_fixed_points[i] for i in range(n_components)}

df_fixed_pts = pd.DataFrame(np.zeros([n_simuls*n_neurons, 2], dtype=np.int64), 
                            index=pd.MultiIndex.from_product([list(range(n_simuls)), 
                                        list(range(n_neurons))], names=["Simulation", "Neuron"]), 
                            columns=pd.Index(["Annotation", "Distance"], name="Fixed point"))
df_fixed_pts = df_fixed_pts.astype({'Distance': 'float64'})

for i in range(n_simuls):
    for j in range(n_neurons):
        annot, dista = annotate_fixed_point(late_time_series[i][:, j], fixed_pts_dict=dict_fixed_points)
        df_fixed_pts.loc[(i, j), "Annotation"] = annot
        df_fixed_pts.loc[(i, j), "Distance"] = dista

In [ ]:
late_time_series[0].mean(axis=0)

In [ ]:
df_fractions = df_fixed_pts["Annotation"].reset_index().groupby(["Simulation", "Annotation"]).count().unstack("Annotation") / n_neurons * 100
df_fractions.columns = df_fractions.columns.droplevel(0)
df_fractions.columns.name = "Fixed point"

In [ ]:
# Bar graph showing this?
fig, axes = plt.subplots(5, 2)
fig.set_size_inches(2*1.5, 5*1.3)
for i in df_fractions.index:
    axes.flat[i].pie(df_fractions.loc[i].values, autopct='%d%%', shadow=False)
fig.tight_layout(w_pad=0.3, h_pad=0.3)
#fig.savefig("figures/three_odors/pie_charts_neuron_distributions_different_runs.pdf", 
#            transparent=True, bbox_inches="tight")
plt.show()
plt.close()